# General Usage

In [1]:
from pprint import pprint
import tdameritrade as td
import os
import json

## Authentication



In [2]:
# get refreshtoken from file
refreshtoken = open(os.path.expanduser('~/.r_token'), 'r')
line = refreshtoken.readline()
refreshtoken.close()

client_id = os.getenv('TDAMERITRADE_CLIENT_ID')
account_id = os.getenv('TDAMERITRADE_ACCOUNT_ID')

# authenticate and initialize TD Client
td_client = td.TDClient(client_id=client_id, refresh_token=line)



## API Examples

In [3]:
# shows private account info
#pprint(td_client.accounts())

### Getting Symbol Data

In [4]:
pprint(td_client.search('aapl'))

{'AAPL': {'assetType': 'EQUITY',
          'cusip': '037833100',
          'description': 'Apple Inc. - Common Stock',
          'exchange': 'NASDAQ',
          'symbol': 'AAPL'}}


In [5]:
pprint(td_client.quote('aapl'))

{'AAPL': {'52WkHigh': 145.09,
          '52WkLow': 53.1525,
          'askId': 'Q',
          'askPrice': 136.24,
          'askSize': 100,
          'assetMainType': 'EQUITY',
          'assetType': 'EQUITY',
          'bidId': 'Q',
          'bidPrice': 136.23,
          'bidSize': 100,
          'bidTick': ' ',
          'closePrice': 136.76,
          'cusip': '037833100',
          'delayed': False,
          'description': 'Apple Inc. - Common Stock',
          'digits': 4,
          'divAmount': 0.82,
          'divDate': '2021-02-05 00:00:00.000',
          'divYield': 0.6,
          'exchange': 'q',
          'exchangeName': 'NASD',
          'highPrice': 0.0,
          'lastId': 'D',
          'lastPrice': 136.24,
          'lastSize': 0,
          'lowPrice': 0.0,
          'marginable': True,
          'mark': 136.24,
          'markChangeInDouble': -0.52,
          'markPercentChangeInDouble': -0.3802,
          'nAV': 0.0,
          'netChange': -0.52,
          'netPerce

In [6]:
pprint(td_client.instrument('037833100'))

[{'assetType': 'EQUITY',
  'cusip': '037833100',
  'description': 'Apple Inc. - Common Stock',
  'exchange': 'NASDAQ',
  'symbol': 'AAPL'}]


In [7]:
import pandas as pd
pd.DataFrame(td_client.history('AAPL')['candles']).head()

,open,high,low,close,volume,datetime
0,141.92,142.33,141.89,142.33,57702,1611576000000
1,142.35,142.50,142.30,142.40,25874,1611576060000
2,142.40,142.46,142.35,142.40,55393,1611576120000
3,142.40,142.40,142.31,142.35,20794,1611576180000
4,142.35,142.45,142.30,142.35,24813,1611576240000


### Getting Option Data

In [8]:
options = td_client.options('AAPL')

In [9]:
expiry_dates = list(options['putExpDateMap'].keys())
expiry = expiry_dates[0]
print(f"Expiry Date: {expiry}")

strike_prices = list(options['putExpDateMap'][expiry].keys())
strike = strike_prices[0]
print(f"Strike Price: {strike}")

pprint(options['putExpDateMap'][expiry][strike])

Expiry Date: 2021-02-12:4
Strike Price: 65.0
[{'ask': 0.01,
  'askSize': 135,
  'bid': 0.0,
  'bidAskSize': '0X135',
  'bidSize': 0,
  'closePrice': 0.0,
  'daysToExpiration': 4,
  'deliverableNote': '',
  'delta': 0.0,
  'description': 'AAPL Feb 12 2021 65 Put (Weekly)',
  'exchangeName': 'OPR',
  'expirationDate': 1613163600000,
  'expirationType': 'S',
  'gamma': 0.0,
  'highPrice': 0.0,
  'inTheMoney': False,
  'isIndexOption': None,
  'last': 0.01,
  'lastSize': 0,
  'lastTradingDay': 1613178000000,
  'lowPrice': 0.0,
  'mark': 0.01,
  'markChange': 0.0,
  'markPercentChange': 4900.0,
  'mini': False,
  'multiplier': 100.0,
  'netChange': 0.01,
  'nonStandard': False,
  'openInterest': 28,
  'openPrice': 0.0,
  'optionDeliverablesList': None,
  'percentChange': 9900.0,
  'putCall': 'PUT',
  'quoteTimeInLong': 1612558799661,
  'rho': 0.0,
  'settlementType': ' ',
  'strikePrice': 65.0,
  'symbol': 'AAPL_021221P65',
  'theoreticalOptionValue': 0.0,
  'theoreticalVolatility': 29.0,
 

## Getting Account Positions

In [11]:
# PRIVATE account info
# pprint(td_client.accounts(positions=True))

## Placing an Order

### Creating order body for Place Order API

In [14]:
from tdameritrade.orders.order_builder import build_buy_market_stock_order

# Prints function docstring
# build_buy_market_stock_order?

In [15]:
symbol = "HUSA"  # Cheap stock for example
quantity = 1
order = build_buy_market_stock_order(symbol, quantity)

In [16]:
pprint(order.asdict())

{'duration': 'DAY',
 'orderLegCollection': [{'instruction': 'BUY',
                         'instrument': {'assetType': 'EQUITY',
                                        'symbol': 'HUSA'},
                         'quantity': 1}],
 'orderStrategyType': 'SINGLE',
 'orderType': 'MARKET',
 'session': 'NORMAL'}


### Submitting Order

In [18]:
# Without this, it fails with a 415.
# This may screw up other API calls, so it may have to be reset if you want to try other ones.
td_client.session.headers = {
    "Content-Type": "application/json"
}

In [19]:
# This will fail to return because the place order API does not return a JSONserializable response on success
# BUT the order will still go through.  Checked on the site.  You can also check your active orders below.

try:
    response = td_client.placeOrder(account_id, order.json())
    pprint(response)
except json.decoder.JSONDecodeError as err:
    print(err)


ValidationError: -1: An unknown error occurred

### Getting out Orders

In [12]:
# Load account info to get account ID for get orders
#td_client.accounts()

#account_id = td_client.accountIds[0]

In [13]:
# Print out orders
my_orders = td_client.orders(accountId=account_id)
pprint(my_orders)

[]


### Deleting Orders

In [19]:
# This deletes all WORKING orders

# for order in my_orders:
#     if order["status"] == Status.WORKING and order["cancelable"]:
#         response = td_client.cancelOrder(account_id, order["orderId"])
#         pprint(response)